In [54]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [107]:

REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

class Dice():
    IMG_SIZE = 50
    TRAINING_DIR = "finalRealTrain"
    SAMPLES_NUM = 100

    training_data = []

    counter = [0 for i in range(7)]
    

    def make_training_data(self):
        for f in tqdm(os.listdir(self.TRAINING_DIR)):
            if "jpg" in f:
#                 print(f)
                path = os.path.join(self.TRAINING_DIR, f)
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                try:
                    c = int(f[-5]) - 1
                except Exception as e:
                    c = int(f[-13]) - 1
                if self.counter[c] < self.SAMPLES_NUM:
                    ans = np.eye(7)[c]
                    self.training_data.append([np.array(img), ans])
                    self.counter[c] += 1


        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        for ind, value in enumerate(self.counter):
            print(ind, value)


if REBUILD_DATA:
    dice = Dice()
    dice.make_training_data()


training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))
print(training_data[0])

100%|██████████| 800/800 [00:02<00:00, 330.86it/s]

0 86
1 80
2 84
3 92
4 100
5 82
6 100
624
[array([[ 73,  81,  79, ...,  46,  48,  38],
        [ 76,  78,  83, ...,  42,  40,  33],
        [ 77,  76,  80, ...,  42,  42, 151],
        ...,
        [ 98,  96, 106, ..., 109, 104, 105],
        [ 78,  76,  85, ..., 110,  67,  84],
        [ 37,  85,  92, ..., 109, 111,  93]], dtype=uint8)
 array([0., 0., 0., 0., 0., 1., 0.])]


In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 64, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(64, 128, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(128, 256, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 7) # 512 in, 6 out bc we're doing 6 classes (dice)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
net

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

In [62]:
training_data[0][0].reshape(-1, 50, 50)

array([[[ 76,  84,  85, ..., 204, 201, 200],
        [ 77,  73,  85, ..., 190,  77, 141],
        [ 77,  77,  75, ..., 191,  96,  84],
        ...,
        [ 95,  80,  37, ..., 103, 110,  47],
        [ 74, 102,  94, ..., 114,  77,  46],
        [ 31,  97,  96, ..., 110, 107,  43]]], dtype=uint8)

In [109]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])
len(X)

624

In [110]:
VAL_PCT = 0.05  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

31


In [111]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [112]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=1e-3)
loss_function = nn.MSELoss()

In [114]:
BATCH_SIZE = 30
# EPOCHS = int(1e6)
EPOCHS = 100

for epoch in range(EPOCHS):
    for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]
#         print(batch_X[0])
#         break
        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss}")
    
    
correct = 0

total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))


Epoch: 0. Loss: 7.073007145663723e-05
Epoch: 1. Loss: 1.7597703845240176e-05
Epoch: 2. Loss: 8.430925663560629e-05
Epoch: 3. Loss: 4.340196028351784e-05
Epoch: 4. Loss: 5.544480882235803e-05
Epoch: 5. Loss: 0.0007748758071102202
Epoch: 6. Loss: 0.0004633853386621922
Epoch: 7. Loss: 7.452373392879963e-05
Epoch: 8. Loss: 3.267923602834344e-05
Epoch: 9. Loss: 4.803792035090737e-05
Epoch: 10. Loss: 0.0014456268399953842
Epoch: 11. Loss: 1.612061714695301e-05
Epoch: 12. Loss: 0.0016440736362710595
Epoch: 13. Loss: 2.4960949303931557e-05
Epoch: 14. Loss: 6.13028823863715e-05
Epoch: 15. Loss: 0.00021851889323443174
Epoch: 16. Loss: 1.3020036931266077e-05
Epoch: 17. Loss: 3.432903758948669e-05
Epoch: 18. Loss: 1.2182420505268965e-05
Epoch: 19. Loss: 7.14188990968978e-06
Epoch: 20. Loss: 4.064337190357037e-06
Epoch: 21. Loss: 1.4278665730671491e-05
Epoch: 22. Loss: 9.338663221569732e-05
Epoch: 23. Loss: 0.0001127320429077372
Epoch: 24. Loss: 5.548338594962843e-05
Epoch: 25. Loss: 6.081264473323

100%|██████████| 31/31 [00:00<00:00, 548.69it/s]

Accuracy:  0.968


In [115]:
correct = 0
total = 0
counter = [0 for i in range(7)]
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)
        counter[predicted_class] += 1

        if predicted_class == real_class:
            correct += 1
        total += 1
print(counter)
print("Accuracy: ", round(correct/total, 3))

100%|██████████| 31/31 [00:00<00:00, 415.46it/s]

[6, 6, 4, 5, 5, 5, 0]
Accuracy:  0.968


In [103]:
torch.save(net, "finalNet.pt")